In [38]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

import torch
import json
import pandas as pd

In [39]:
results = torch.load("/cluster/dataset/midatams/inf-extr/results/medication/medication_outlines_Llama2-MedTuned-13b_4bit_few_shot_instruction.pt")

In [40]:
dfs = []
for idx, (answer, text) in enumerate(zip(results["model_answers"], results["text"])):
    answer = json.loads(answer)
    medications = answer["medications"]
    for med in medications:
        med["text"] = text
        med["id"] = idx
        dfs.append((med))
results = pd.DataFrame(dfs)

In [41]:
results.to_csv(paths.RESULTS_PATH/"medication/labels.csv")